<a href="https://colab.research.google.com/github/ekaterinatao/NER_biomed_domain/blob/main/%D0%92%D0%9A%D0%A0_nerel_bio_base.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Установка зависимостей

In [34]:
!pip install git+https://github.com/nicolay-r/arekit-ss.git@0.24.0
!pip install datasets evaluate seqeval -qqq
!pip install accelerate -U -qqq
!pip install wandb -qqq

  Cloning https://github.com/nicolay-r/arekit-ss.git (to revision 0.24.0) to /tmp/pip-req-build-e3bmajzk
  Running command git clone --filter=blob:none --quiet https://github.com/nicolay-r/arekit-ss.git /tmp/pip-req-build-e3bmajzk
  Running command git checkout -b 0.24.0 --track origin/0.24.0
  Switched to a new branch '0.24.0'
  Branch '0.24.0' set up to track remote branch '0.24.0' from 'origin'.
  Resolved https://github.com/nicolay-r/arekit-ss.git to commit b5f66fc8aab8b342386e3e6eb60f1d759573b9d3
  Preparing metadata (setup.py) ... done


In [1]:
import numpy as np
import pandas as pd
import re
from pathlib import Path

import torch
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer
from transformers import DataCollatorForTokenClassification
from transformers import AutoModelForTokenClassification
from transformers import TrainingArguments, Trainer
from transformers import pipeline
import evaluate

import warnings
warnings.filterwarnings("ignore")

### Скачивание датасета [NEREL-BIO](https://github.com/nerel-ds/NEREL-BIO)

In [2]:
!python -m arekit.download_data

100% 53.5k/53.5k [00:00<00:00, 23.3MB/s]
100% 705k/705k [00:00<00:00, 32.0MB/s]
100% 54.2k/54.2k [00:00<00:00, 6.36MB/s]
100% 57.4k/57.4k [00:00<00:00, 5.15MB/s]
100% 737/737 [00:00<00:00, 384kB/s]
100% 3.40M/3.40M [00:00<00:00, 4.73MB/s]
100% 3.46M/3.46M [00:01<00:00, 2.94MB/s]
100% 27.6M/27.6M [00:01<00:00, 14.7MB/s]
100% 146M/146M [00:05<00:00, 24.8MB/s]
100% 52.1M/52.1M [00:03<00:00, 16.1MB/s]
100% 321M/321M [00:13<00:00, 23.0MB/s]
73.6kB [00:00, 1.14MB/s]
73.6kB [00:00, 1.19MB/s]
73.6kB [00:00, 433kB/s]
100% 2.75M/2.75M [00:00<00:00, 20.2MB/s]
100% 3.20M/3.20M [00:00<00:00, 13.4MB/s]
100% 2.59M/2.59M [00:01<00:00, 2.21MB/s]


In [3]:
!unzip /root/.arekit/nerel-bio-v1_0.zip

Archive:  /root/.arekit/nerel-bio-v1_0.zip
   creating: dev/
  inflating: dev/26036068_ru.txt     
  inflating: dev/27271827_ru.txt     
  inflating: dev/24347732_ru.ann     
  inflating: dev/27296265_ru.txt     
  inflating: dev/27296118_ru.txt     
  inflating: dev/27271721_ru.txt     
  inflating: dev/27296006_ru.ann     
  inflating: dev/25177072_ru.txt     
  inflating: dev/25378754_ru.ann     
  inflating: dev/25378759_ru.txt     
  inflating: dev/25378728_ru.txt     
  inflating: dev/27274597_ru.txt     
  inflating: dev/27296263_ru.txt     
  inflating: dev/27271832_ru.ann     
  inflating: dev/25378740_ru.ann     
  inflating: dev/27271713_ru.txt     
  inflating: dev/24391299_ru.txt     
  inflating: dev/27296534_ru.txt     
  inflating: dev/24391296_ru.txt     
  inflating: dev/27296002_ru.ann     
  inflating: dev/27296001_ru.ann     
  inflating: dev/27296003_ru.ann     
  inflating: dev/25378728_ru.ann     
  inflating: dev/24700993_ru.txt     
  inflating: dev/27296263_r

In [4]:
with open('dev/24700993_ru.txt', 'r') as f:
   example_txt = f.read()

example_txt

'Цель\nИзучить биомаркеры поступления метилртути (MeHg) у женщин и детей из группы населения, потребляющего морепродукты, в мировом масштабе и охарактеризовать сравнительный риск отдаленного нейротоксического действия на плод.\nМетоды\nБыл проведен поиск опубликованной литературы, в которой сообщалось об общем содержании ртути (Hg) в волосах и крови женщин и детей.\nЭти биомаркеры являются подтвержденными репрезентативными индикаторами содержания MeHg – нейротоксина, обнаруживаемого главным образом в морепродуктах.\nПосле отбора биомаркеры среднего и высокого уровней были разделены по контексту потребления морепродуктов и сгруппированы по категориям.\nМедианные значения распределений биомаркеров для среднего и высокого уровней сравнивались с контрольным уровнем, установленным объединенным экспертным комитетом Продовольственной и сельскохозяйственной организации ООН (ФАО) и Всемирной организацией здравоохранения (ВОЗ).\nРезультаты\nКритериям выбора соответствовали 164 исследования женщи

In [2]:
example_ann = pd.read_csv(
    'dev/24700993_ru.ann', sep='\t', header=None, names=['type', 'tag', 'word']
)
example_ann.head(10)

,type,tag,word
0,T1,ORGANIZATION 793 870,экспертным комитетом Продовольственной и сельс...
1,T2,ORGANIZATION 814 870,Продовольственной и сельскохозяйственной орган...
2,T3,ORGANIZATION 867 870,ООН
3,T4,ORGANIZATION 872 875,ФАО
4,T5,ORGANIZATION 919 922,ВОЗ
5,T6,NUMBER 969 972,164
6,T7,NUMBER 1004 1006,43
7,T8,LOCATION 1359 1366,Арктике
8,T10,LOCATION 1409 1437,Западной части Тихого океана
9,T11,NUMBER 1533 1536,две


### Подготовка данных к загрузке в DataLoader

In [3]:
tags = ("""AGE
AWARD
CITY
COUNTRY
CRIME
DATE
DISO
DISTRICT
EVENT
FACILITY
FAMILY
IDEOLOGY
LANGUAGE
LAW
LOCATION
MONEY
NATIONALITY
NUMBER
ORDINAL
ORGANIZATION
PENALTY
PERCENT
PERSON
PRODUCT
PROFESSION
RELIGION
STATE_OR_PROVINCE
TIME
WORK_OF_ART
ANATOMY
LIVB
ACTIVITY
CHEM
DEVICE
PHYS
MEDPROC
SCIPROC
LABPROC
FINDING
MENTALPROC
GENE
FOOD
INJURY_POISONING
ADMINISTRATION_ROUTE
HEALTH_CARE_ACTIVITY""").split()

In [4]:
tag_to_id = {tag: i for i, tag in enumerate(tags)}
id_to_tag = {i: tag for i, tag in enumerate(tags)}

In [5]:
TRAIN_DIR = Path('./train/')
DEV_DIR = Path('./train/')
TEST_DIR = Path('./train/')
train_files = list(TRAIN_DIR.rglob('*.ann'))
dev_files = list(DEV_DIR.rglob('*.ann'))
test_files = list(TEST_DIR.rglob('*.ann'))
print(f'train: {len(train_files)}, dev: {len(dev_files)}, test: {len(test_files)}')

train: 612, dev: 612, test: 612


In [6]:
def preprocessing(files):
    preprocessed_data = []
    for path in files:
        file_df = pd.read_csv(path, header=None, sep='\t', names=['type', 'tag', 'word'])
        id = re.findall(r'\d+', str(path))

        temp_list_word = []
        temp_list_ner_tags = []

        for i, val in enumerate(file_df['type'].values):
            try:
                if re.findall(r'T\d+', val)[0][0] == 'T':
                    tag = re.findall(r'\w+', file_df.loc[i, 'tag'])[0]
                    temp_list_word.append(file_df.loc[i, 'word'])
                    temp_list_ner_tags.append(tag_to_id[tag])
            except: pass

        preprocessed_data.append({
            'id': id,
            'words': temp_list_word,
            'ner_tags': temp_list_ner_tags
        })

    return preprocessed_data

In [7]:
dataset = DatasetDict({
    'train': Dataset.from_pandas(pd.DataFrame(preprocessing(train_files))),
    'valid': Dataset.from_pandas(pd.DataFrame(preprocessing(dev_files))),
    'test': Dataset.from_pandas(pd.DataFrame(preprocessing(test_files)))
})
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'words', 'ner_tags'],
        num_rows: 612
    })
    valid: Dataset({
        features: ['id', 'words', 'ner_tags'],
        num_rows: 612
    })
    test: Dataset({
        features: ['id', 'words', 'ner_tags'],
        num_rows: 612
    })
})

In [8]:
for token, ner_tag in zip(dataset['train'][0]['words'], dataset['train'][0]['ner_tags']):
    print(f'{token:_<100}{ner_tag}')

ДС__________________________________________________________________________________________________6
СКП_________________________________________________________________________________________________33
42__________________________________________________________________________________________________17
ДС__________________________________________________________________________________________________6
21__________________________________________________________________________________________________17
21__________________________________________________________________________________________________17
30,4±25,6%__________________________________________________________________________________________21
17,0±19,4___________________________________________________________________________________________21
16,6±21,6%__________________________________________________________________________________________21
1-й________________________________________________________________________

___
### Токенизация с помощью модели [RuBERT](https://huggingface.co/DeepPavlov/rubert-base-cased)

In [9]:
tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased")

In [13]:
example = dataset["train"][0]
tokenized_input = tokenizer(example["words"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])

print(f'Token len from BERT tokenizer__{len(tokens)}')
print(f'Token len initial tokens__{len(example["words"])}\n')

print(' '.join(tokens))

Token len from BERT tokenizer__521
Token len initial tokens__154

[CLS] ДС СКП 42 ДС 21 21 30 , 4 ##± ##25 , 6 % 17 , 0 ##± ##19 , 4 16 , 6 ##± ##21 , 6 % 1 - й 80 % 90 % репара ##тив ##ных процессов мягких тканях нижних конечностей пациентов содержащих колла ##ген повяз ##ок гист ##ологические иммуног ##исто ##хим ##ические марке ##ры репара ##тив ##ных процессов мягких тканях пациентов ран ##евых колла ##гена пациента нейро ##ишем ##ической формами синдрома ДС синдрома ДС пациентов ран ##евых ч ##реск ##ож ##ной окс ##иметр ##ии колла ##ген ##ом гист ##ологического исследования тканей ран ##евых дефектов уменьшение от ##ека воспал ##ительного инф ##ильт ##рата зрел ##ой гранул ##я ##ционной ткани иммуног ##исто ##хим ##ического исследования макроф ##агов дер ##ме матрик ##с ##ной металло ##прот ##еи ##наз ##ы - 9 репара ##тив ##ных процессов мягких тканях больных колла ##ген ##ом ран перф ##узии тканей воспаления раны пролиф ##ерации эпители ##зац ##ии гист ##ологического исследовани

In [10]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["words"], truncation=True, max_length=512, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [11]:
tokenized_dataset = dataset.map(tokenize_and_align_labels, batched=True)
tokenized_dataset

Map:   0%|          | 0/612 [00:00<?, ? examples/s]

Map:   0%|          | 0/612 [00:00<?, ? examples/s]

Map:   0%|          | 0/612 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'words', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 612
    })
    valid: Dataset({
        features: ['id', 'words', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 612
    })
    test: Dataset({
        features: ['id', 'words', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 612
    })
})

In [12]:
df = tokenized_dataset['train']
pd.DataFrame({
    'id': df['id'], 'words': df['words'], 'ner_tags': df['ner_tags'],
    'input_ids': df['input_ids'], 'token_type_ids': df['token_type_ids'],
    'attention_mask': df['attention_mask'], 'labels': df['labels']
}).head(5)

,id,words,ner_tags,input_ids,token_type_ids,attention_mask,labels
0,[26978178],"[ДС, СКП, 42, ДС, 21, 21, 30,4±25,6%, 17,0±19,...","[6, 33, 17, 6, 17, 17, 21, 21, 21, 18, 21, 21,...","[101, 41211, 37776, 11788, 41211, 6401, 6401, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, 6, 33, 17, 6, 17, 17, 21, -100, -100, -..."
1,[26271703],"[29, 13-16 лет, ХТ, 2-13 лет, 30, 12-16 лет, 8...","[17, 0, 35, 0, 17, 0, 17, 29, 22, 35, 22, 29, ...","[101, 7688, 4834, 130, 4370, 3422, 819, 814, 1...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, 17, 0, -100, -100, -100, 35, -100, 0, -..."
2,[27030437],"[81, 81, 2, 1-й, 41, 41, 0,5 мл, 10 дней, 40, ...","[17, 17, 17, 18, 17, 17, 17, 5, 17, 17, 18, 21...","[101, 12836, 12836, 140, 138, 130, 859, 11861,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, 17, 17, 17, 18, -100, -100, 17, 17, 17,..."
3,[26977790],"[162, 152, 6%, Краниотомия, неврологических на...","[17, 17, 21, 35, 6, 34, 35, 35, 22, 38, 42, 29...","[101, 22150, 19103, 148, 114, 59349, 33789, 37...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, 17, 17, 21, -100, 35, -100, -100, -100,..."
4,[26978046],"[8, 19, 18-44 лет, АГ, две, межъядерных интерн...","[17, 17, 0, 6, 17, 29, 32, 32, 32, 29, 22, 6, ...","[101, 152, 1653, 2810, 130, 11638, 3422, 71882...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, 17, 17, 0, -100, -100, -100, 6, 17, 29,..."


### Обучение модели с различными гиперпараметрами (`learning rate`, `decay weight`)

In [13]:
seqeval = evaluate.load("seqeval")

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [tags[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [tags[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [14]:
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [16]:
LR_VALUES = (2e-5, 5e-5)
DECAY_VALUES = (1e-4, 0.1)

In [17]:
torch.cuda.empty_cache()

In [18]:
for i, learning_rate in enumerate(LR_VALUES):
    for j, weight_decay in enumerate(DECAY_VALUES):

            model = AutoModelForTokenClassification.from_pretrained(
                "DeepPavlov/rubert-base-cased", num_labels=45, id2label=id_to_tag, label2id=tag_to_id)
            print(f'Log: training for l_r:{learning_rate}, w_d:{weight_decay}...')

            training_args = TrainingArguments(
                output_dir="token_class_model",
                learning_rate=learning_rate,
                per_device_train_batch_size=4,
                per_device_eval_batch_size=4,
                num_train_epochs=10,
                weight_decay=weight_decay,
                evaluation_strategy="epoch",
                push_to_hub=False,
                save_strategy="no",
                group_by_length=True,
                warmup_ratio=0.1,
                optim="adamw_torch",
                lr_scheduler_type="cosine",
            )

            trainer = Trainer(
                model=model,
                args=training_args,
                train_dataset=tokenized_dataset["train"],
                eval_dataset=tokenized_dataset["valid"],
                tokenizer=tokenizer,
                data_collator=data_collator,
                compute_metrics=compute_metrics,
            )

            trainer.train()

Some weights of BertForTokenClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Log: training for l_r:2e-05, w_d:0.0001...


You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,1.069634,0.622016,0.568100,0.593837,0.732727
2,No log,0.541432,0.776968,0.743154,0.759685,0.853728
3,No log,0.379992,0.836747,0.802283,0.819152,0.895034
4,1.189600,0.273499,0.876655,0.851553,0.863921,0.923797
5,1.189600,0.203992,0.904388,0.885047,0.894613,0.942289
6,1.189600,0.157178,0.927800,0.909015,0.918312,0.956070
7,0.285600,0.132090,0.938441,0.921291,0.929787,0.962721
8,0.285600,0.115018,0.947133,0.933428,0.940230,0.968411
9,0.285600,0.107263,0.951296,0.938482,0.944846,0.970775
10,0.155700,0.106440,0.951656,0.938705,0.945136,0.970978


Some weights of BertForTokenClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

Log: training for l_r:5e-05, w_d:0.0001...


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.835297,0.701723,0.655085,0.677602,0.787095
2,No log,0.414513,0.817718,0.792896,0.805116,0.883913
3,No log,0.271829,0.872975,0.857579,0.865209,0.923022
4,0.973500,0.193857,0.903156,0.889435,0.896243,0.942880
5,0.973500,0.119605,0.940117,0.930456,0.935261,0.965713
6,0.973500,0.084148,0.957983,0.949203,0.953573,0.975855
7,0.185500,0.062536,0.968354,0.960312,0.964316,0.981970
8,0.185500,0.041286,0.980974,0.975199,0.978078,0.988953
9,0.185500,0.033698,0.984504,0.979309,0.981900,0.990911
10,0.061700,0.032481,0.985008,0.979892,0.982443,0.991207


Some weights of BertForTokenClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Log: training for l_r:5e-05, w_d:0.1...


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.836159,0.701467,0.654613,0.677231,0.786873
2,No log,0.416116,0.821398,0.791924,0.806391,0.884523
3,No log,0.275188,0.873558,0.849636,0.861431,0.920638
4,0.974300,0.184972,0.914569,0.895240,0.904801,0.947554
5,0.974300,0.130003,0.935897,0.922485,0.929143,0.962074
6,0.974300,0.089545,0.955050,0.946509,0.950760,0.973971
7,0.189500,0.063513,0.969523,0.960368,0.964924,0.981841
8,0.189500,0.044781,0.978652,0.971449,0.975037,0.987383
9,0.189500,0.035688,0.983236,0.977365,0.980292,0.990043
10,0.064100,0.034339,0.983656,0.977809,0.980724,0.990228


___
Лучшими оказались гиперпараметры: `l_r`= 5e-05, `w_d` = 0.0001  
Веса модели сохранены в [репозитории](https://huggingface.co/ekaterinatao/NEREL-BIO_RuBERT_base)  
Отчет обучения лучшей модели на [wandb](https://wandb.ai/taoea/NEREL-BIO_RuBERT_base/reports/NEREL-BIO-RuBert-base--Vmlldzo2MTE3ODQ1)

In [22]:
from huggingface_hub import notebook_login
notebook_login()

In [20]:
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [21]:
import os
os.environ["WANDB_PROJECT"]="NEREL-BIO_RuBERT_base"

In [23]:
model = AutoModelForTokenClassification.from_pretrained(
    "DeepPavlov/rubert-base-cased", num_labels=45, id2label=id_to_tag, label2id=tag_to_id)

repo_id = "ekaterinatao/NEREL-BIO_RuBERT_base"

training_args = TrainingArguments(
    output_dir=repo_id,
    learning_rate=5e-05,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=10,
    weight_decay=0.0001,
    evaluation_strategy="steps",
    logging_steps=50,
    push_to_hub=True,
    report_to="wandb",
    run_name="first_run",
    save_strategy="no",
    group_by_length=True,
    warmup_ratio=0.1,
    optim="adamw_torch",
    lr_scheduler_type="cosine",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["valid"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

Some weights of BertForTokenClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


wandb: Currently logged in as: taoea. Use `wandb login --relogin` to force relogin


Step,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
50,3.073900,2.366506,0.184240,0.087013,0.118202,0.355815
100,1.809200,1.173213,0.577427,0.513664,0.543683,0.701119
150,1.103200,0.815456,0.695465,0.654613,0.674421,0.787446
200,0.833800,0.605756,0.751199,0.717630,0.734031,0.835532
250,0.706800,0.479649,0.791195,0.762651,0.776661,0.866253
300,0.583900,0.428521,0.815994,0.790952,0.803278,0.883248
350,0.443000,0.358890,0.830447,0.827057,0.828748,0.898766
400,0.431200,0.307218,0.856924,0.839527,0.848136,0.913341
450,0.386200,0.280189,0.869300,0.850275,0.859682,0.919826
500,0.290000,0.237869,0.887375,0.875299,0.881295,0.932387


TrainOutput(global_step=1530, training_loss=0.3936850890614628, metrics={'train_runtime': 459.7931, 'train_samples_per_second': 13.31, 'train_steps_per_second': 3.328, 'total_flos': 845031394630800.0, 'train_loss': 0.3936850890614628, 'epoch': 10.0})

In [33]:
trainer.save_model(repo_id)

model.safetensors:   0%|          | 0.00/709M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/4.60k [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

### Оценка качества на тестовой выборке

In [24]:
predictions = trainer.predict(test_dataset=tokenized_dataset["test"])

In [25]:
predictions.metrics

{'test_loss': 0.03074502944946289,
 'test_precision': 0.9854285077454586,
 'test_recall': 0.9823085041381991,
 'test_f1': 0.9838660324348383,
 'test_accuracy': 0.9917424074484593,
 'test_runtime': 12.9438,
 'test_samples_per_second': 47.281,
 'test_steps_per_second': 11.82}

In [26]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'device is: {device}')

device is: cuda


Оценка модели на абстракте, которого не было в тестовом наборе

In [27]:
ner_bio = pipeline("ner", model=model, tokenizer=tokenizer, device=device)

In [31]:
abstract = """Цель. Оценить выживаемость у пациентов с болезнью Фабри (БФ) в зависимости от вида заместительной почечной терапии, и определить роль диализного скрининга в ранней диагностике БФ у родственников.
Материалы и методы. В исследование включали взрослых (старше 18 лет) пациентов с подтвержденным диагнозом БФ. Терминальная стадия хронической почечной недостаточности (тХПН) диагностировали в соответствии с рекомендациями Научного общества нефрологов России (2016) и KDIGO (2012). На основании опроса пробандов выявляли его родственников, которые могли унаследовать мутантный ген.
Результаты. У 50 (24,9%) из 201 обследованных пациентов с БФ диагностирована тХПН, в том числе у 48 (40%) из 120 мужчин и 2 (2,7%) из 81 женщин. Оценка кумулятивной частоты методом Каплана-Майера демонстрирует выраженное увеличение частоты регистрации тХПН к возрасту 20-30 лет, а к возрасту 50 лет ожидаемое количество пациентов с тХПН составляет 95%. Пяти из 50 больных с тХПН была выполнена трансплантация почки, в среднем, через 17 месяцев (диапазон от 7 до 70 месяцев) после инициации лечения гемодиализом. Умерло 15 (30%) из 50 пациентов, получавших лечение гемодиализом. Все умершие пациенты были мужского пола. Медиана возраста на момент летального исхода составила 45 (39; 58) лет. Среди пациентов, которым проведена трансплантация почки, летальных исходов зарегистрировано не было. У 44 (88%) из 50 пациентов диагноз БФ установлен, в среднем, через 1 год (диапазон от 0 до 12 лет) после начала лечения программным гемодиализом, в том числе у одного пациента – после трансплантации почки. Среди 44 пробандов, выявленных при всероссийском диализном скрининге, проведен семейный скрининг. Патогенная мутация в гене GLA диагностирована у 89 (57%) из 156 обследованных родственников диализных пробандов, в том числе у 18 детей моложе 18 лет, клинические проявления БФ имелись у 48 родственников. У 80,4% обследованных родственников диализных пробандов обнаружено поражение почек, преимущественно на ранних стадиях.
Заключение. ТХПН нередкое осложнение БФ, ассоциированное с неблагоприятным прогнозом. Однако диализный скрининг –  эффективный способ выявления пробандов с БФ, открывающий возможность установить диагноз БФ у родственников на ранних стадиях, когда лечение наиболее эффективно.
"""

In [32]:
ner_bio(abstract)

[{'entity': 'SCIPROC',
  'score': 0.6651787,
  'index': 1,
  'word': 'Цель',
  'start': 0,
  'end': 4},
 {'entity': 'SCIPROC',
  'score': 0.49190542,
  'index': 2,
  'word': '.',
  'start': 4,
  'end': 5},
 {'entity': 'FINDING',
  'score': 0.7045247,
  'index': 3,
  'word': 'Оцен',
  'start': 6,
  'end': 10},
 {'entity': 'FINDING',
  'score': 0.75048,
  'index': 4,
  'word': '##ить',
  'start': 10,
  'end': 13},
 {'entity': 'PHYS',
  'score': 0.86772174,
  'index': 5,
  'word': 'выжив',
  'start': 14,
  'end': 19},
 {'entity': 'PHYS',
  'score': 0.94870025,
  'index': 6,
  'word': '##аемость',
  'start': 19,
  'end': 26},
 {'entity': 'FINDING',
  'score': 0.44612923,
  'index': 7,
  'word': 'у',
  'start': 27,
  'end': 28},
 {'entity': 'PERSON',
  'score': 0.99794656,
  'index': 8,
  'word': 'пациентов',
  'start': 29,
  'end': 38},
 {'entity': 'DISO',
  'score': 0.99812526,
  'index': 9,
  'word': 'с',
  'start': 39,
  'end': 40},
 {'entity': 'DISO',
  'score': 0.9991019,
  'index': 1